# **Laboratorio 7**

- Derek Arreaga - 22537
- Paula Barillas - 22764
- Mónica Salvatierra - 22249

Link del respositorio: https://github.com/FabianKel/ModSim-LAB7

## **Parte 1**

### **Teoría**

**1. Describa con sus propias palabras la diferencia fundamental entre una Entidad de DES y un Agente de MBA. ¿Por qué esta diferencia hace que cada paradigma sea adecuado para tipos de problemas distintos? Proporcione un ejemplo de un problema de modelado para cada uno.**

La diferencia principal está en el tipo de comportamiento que representan. En un modelo de DES (Simulación de Eventos Discretos), una entidad es algo pasivo que pasa por diferentes etapas o procesos del sistema, como si siguiera un recorrido predefinido. Las entidades no deciden nada por sí mismas, simplemente reaccionan a los eventos del sistema. En cambio, en un MBA (Modelado Basado en Agentes), los agentes son activos y autónomos. Cada agente puede tomar decisiones, interactuar con otros y adaptarse a su entorno.
<br>
<br>
DES se usa más en sistemas operativos o logísticos, donde hay flujos de trabajo definidos (por ejemplo, una fila en un banco o una línea de producción). MBA se usa cuando los comportamientos individuales influyen en el resultado global, como en simulaciones de tráfico o en la propagación de enfermedades.
<br>
<br>
- **Ejemplo DES**: modelar el flujo de pacientes en una clínica para analizar tiempos de espera y uso de recursos.
- **Ejemplo MBA**: modelar cómo las personas reaccionan ante una epidemia, considerando decisiones como aislarse o vacunarse.

**2. Explique por qué las colas son una consecuencia inevitable en la mayoría de los sistemas modelados con DES. ¿Qué tipo de información valiosa podemos extraer al analizar la longitud de una cola y el tiempo de espera en ella a lo largo de una simulación?**

 Las colas aparecen porque los recursos disponibles son limitados y las llegadas o demandas son variables. Si en algún momento llegan más entidades de las que pueden ser atendidas, se genera una acumulación en espera. Por eso, en la mayoría de los sistemas modelados con DES, las colas son una consecuencia natural.
<br>
<br>
Analizar la longitud de la cola y el tiempo de espera nos da información muy útil, por ejemplo:

- Identificar los puntos donde se forman cuellos de botella.

- Determinar si el sistema tiene suficientes recursos.

- Evaluar la calidad del servicio y el nivel de satisfacción de los clientes.
<br>
<br>
Por ejemplo, si en una cafetería la cola promedio supera los diez minutos, se podría analizar si conviene agregar otro empleado o modificar el proceso de atención. Así, los datos de la simulación ayudan a tomar decisiones basadas en evidencia.

**3. En clase se mencionaron los recursos móviles, estáticos y portátiles. Describa un escenario (diferente al hospital) que involucre los tres tipos de recursos. Identifique qué elementos serían cada tipo de recurso y cómo interactuarían.**

Un ejemplo podría ser un sistema de reparto de paquetes en una empresa de mensajería, como FedEx o Cargo Expreso. En este caso, se pueden identificar los tres tipos de recursos claramente:
<br>
<br>
- **Recursos estáticos**: los centros de distribución o bodegas principales, donde se clasifican y almacenan los paquetes antes de ser enviados.

- **Recursos móviles**: los camiones o motocicletas que transportan los paquetes desde los centros de distribución hasta los puntos de entrega.

- **Recursos portátiles**: los escáneres, dispositivos móviles o etiquetas electrónicas que los repartidores usan para registrar cada entrega y confirmar la recepción.
<br>
<br>
Estos tres recursos interactúan constantemente. Los centros de distribución (estáticos) asignan rutas y organizan la salida de los vehículos (móviles), mientras que los repartidores usan los escáneres portátiles para actualizar el sistema en tiempo real. Así, se puede analizar la eficiencia del proceso, detectar retrasos, optimizar rutas y mejorar la gestión del inventario y la logística.

**4. Imagine que está modelando la propagación de una enfermedad en una ciudad (un problema clásico de MBA). Ahora, quieres añadir el proceso de vacunación en centros de salud. ¿Cómo podría combinar MBA y DES? ¿Qué parte del modelo sería MBA y cuál DES? ¿Qué información pasaría de un modelo a otro?**

La propagación de una enfermedad en una ciudad es un problema típico de MBA, porque cada persona (agente) puede tener un comportamiento distinto. Por ejmeplo, algunos se aíslan, otros siguen saliendo, algunos se vacunan, etc. Cada decisión individual influye en la dinámica de contagio general. Si ahora se quiere agregar el proceso de vacunación en centros de salud, esa parte se modelaría mejor con DES, ya que implica flujos ordenados de personas que llegan, esperan su turno, son atendidas y luego se retiran.

En este modelo combinado:
<br>
<br>
- El MBA representaría la dinámica de la población: quién se contagia, quién decide vacunarse o moverse por la ciudad.

- El DES modelaría los procesos dentro de los centros de vacunación: tiempo de espera, capacidad de atención, personal disponible, etc.

La información pasaría de un modelo al otro. Por ejemplo, el MBA enviaría al DES el número de personas que decidieron vacunarse, y el DES devolvería los resultados de cuántas fueron atendidas efectivamente. Luego, esa información regresaría al MBA para actualizar la tasa de inmunidad y modificar la propagación de la enfermedad en la población.

## **Referencias**

1. Discrete-Event Modeling. (n.d.). https://www.anylogic.com/use-of-simulation/discrete-event-simulation/

2. What is discrete event simulation and how does it work. (n.d.). Haskoning. https://www.haskoning.com/en/twinn/blogs/2024/what-is-discrete-event-simulation-and-how-does-it-work
